# Converting XML files to CSV 

In [1]:
import pandas as pd
from xml.etree import cElementTree as ElementTree
import xml.etree.ElementTree as et
from tqdm import tqdm
import logging
import re

logging.basicConfig(format='%(asctime)s: %(message)s', filename='analysis.log', level=logging.DEBUG)

## Convert XML files to Python Dictionaries 

In [2]:
class XmlListConfig(list):
    def __init__(self, aList):
        for element in aList:
            if element:
                # treat like dict
                if len(element) == 1 or element[0].tag != element[1].tag:
                    self.append(XmlDictConfig(element))
                # treat like list
                elif element[0].tag == element[1].tag:
                    self.append(XmlListConfig(element))
            elif element.text:
                text = element.text.strip()
                if text:
                    self.append(text)


class XmlDictConfig(dict):
    '''
    Example usage:

    >>> tree = ElementTree.parse('your_file.xml')
    >>> root = tree.getroot()
    >>> xmldict = XmlDictConfig(root)

    Or, if you want to use an XML string:

    >>> root = ElementTree.XML(xml_string)
    >>> xmldict = XmlDictConfig(root)

    And then use xmldict for what it is... a dict.
    '''
    def __init__(self, parent_element):
        if parent_element.items():
            self.update(dict(parent_element.items()))
        for element in parent_element:
            if element:
                # treat like dict - we assume that if the first two tags
                # in a series are different, then they are all different.
                if len(element) == 1 or element[0].tag != element[1].tag:
                    aDict = XmlDictConfig(element)
                # treat like list - we assume that if the first two tags
                # in a series are the same, then the rest are the same.
                else:
                    # here, we put the list in dictionary; the key is the
                    # tag name the list elements all share in common, and
                    # the value is the list itself 
                    aDict = {element[0].tag: XmlListConfig(element)}
                # if the tag has attributes, add those to the dict
                if element.items():
                    aDict.update(dict(element.items()))
                self.update({element.tag: aDict})
            # this assumes that if you've got an attribute in a tag,
            # you won't be having any text. This may or may not be a 
            # good idea -- time will tell. It works for the way we are
            # currently doing XML configuration files...
            elif element.items():
                self.update({element.tag: dict(element.items())})
            # finally, if there are no child tags and no attributes, extract
            # the text
            else:
                self.update({element.tag: element.text})

def get_field(xmldict, path=None, default=None):
    if path is None: return xmldict
    try:
        for p in path:
            # Ugly namespaces are ugly, let's deal with them here
            xmldict = xmldict['{http://www.irs.gov/efile}' + p]
        return xmldict
    except KeyError:
        # If this key does not exist in the dictionary, 
        return default
        # Note: it may be faster to check ```if p not in dict``` rather
        # than set up this try/except block. I think it depends on how
        # often this fails - the more it fails, the worse the try/except does.

In [5]:
tree = et.parse('data/test/201722789349300037_public.xml')
root = tree.getroot()
root

<Element '{http://www.irs.gov/efile}Return' at 0x000002660A022630>

## Analysis

### Analyze one file

This function returns a `1 x n` DataFrame of all relevant data for a single file, which can be appended to a larger DataFrame for multi-file analysis.

In [3]:
def analyze_one(path):
    object_id = re.sub("\D", "", path)
    object_list = [[object_id]]
    object_df = pd.DataFrame(object_list, columns =['Object_ID'])
    tree = et.parse(path)
    root = tree.getroot()
    #Converting from xml to dictionary
    xmldict = XmlDictConfig(root)

    
    ##############Header Information 
    #Date Information
    tax_period_begin = get_field(xmldict, ['ReturnHeader','TaxPeriodBeginDt'])
    
    #Filer Information
    filer_data = get_field(xmldict, ['ReturnHeader', 'Filer'], {})
    ein = get_field(filer_data, ['EIN'])
    business_name = get_field(filer_data, ['BusinessName', 'BusinessNameLine1Txt'])
    city = get_field(filer_data, ['USAddress', 'CityNm'])
    state = get_field(filer_data, ['USAddress', 'StateAbbreviationCd'])
    zipc = get_field(filer_data, ['USAddress', 'ZIPCd'])
    tax_year = get_field(xmldict, ['ReturnHeader', 'TaxYr'])
    filer_list = [[tax_year, tax_period_begin, ein, business_name, city, state, zipc]]
    filer_df = pd.DataFrame(filer_list, columns = ['TaxYr','TaxPeriodBeginDt','EIN', 'BusinessName', 'City', 'State', 'ZIPCd']) 
    
    
    ##############Return Data
    return_data = get_field(xmldict, ['ReturnData', 'IRS990'])
    if return_data is None:
        logging.warning(f'Could not find Return Data for file: {path}')
        return None
    
    #Formation Year
    formation_year = get_field(return_data, ['FormationYr'], "NA")
    
    #State of legal domicile 
    legal_dom = get_field(return_data, ['LegalDomicileStateCd'], "NA")
    
    #Related Org
    related = get_field(return_data, ['RelatedOrganizationsAmt'], 0)

    #GovernmentGrantsAmt
    gov_grants = get_field(return_data, ['GovernmentGrantsAmt'], 0)

    #FederatedCampaignsAmt
    fed_camp = get_field(return_data, ['FederatedCampaignsAmt'], 0)

    #MembershipDuesAmt
    membership = get_field(return_data, ['MembershipDuesAmt'], 0)

    #FundraisingAmt
    fundraising = get_field(return_data, ['FundraisingAmt'], 0)
    
    #NoncashContributionsAmt
    NoncashContributionsAmt = get_field(return_data, ['NoncashContributionsAmt'], 0)
    
    #AllOtherContributionsAmt
    all_other = get_field(return_data, ['AllOtherContributionsAmt'], 0)

    #TotalContributionsAmt
    total_contri = get_field(return_data, ['TotalContributionsAmt'], 0)
    
    #TotalProgramServiceRevenueAmt
    program_service_rev = get_field(return_data, ['TotalProgramServiceRevenueAmt'], 0)
    
    #CYInvestmentIncomeAmt
    CY_investment_income = get_field(return_data, ['CYInvestmentIncomeAmt'], 0)
    
    #CYOtherRevenueAmt
    CYOtherRevenueAmt = get_field(return_data, ['CYOtherRevenueAmt'], 0)
    
    #CYTotalRevenueAmt
    CYTotalRevenueAmt = get_field(return_data, ['CYTotalRevenueAmt'], 0)
    
    #CYTotalExpensesAmt
    CYTotalExpensesAmt = get_field(return_data, ['CYTotalExpensesAmt'], 0)
    
    #CYRevenuesLessExpensesAmt
    CYRevenuesLessExpensesAmt = get_field(return_data, ['CYRevenuesLessExpensesAmt'], 0)
    
    #TotalAssetsEOYAmt
    TotalAssetsEOYAmt = get_field(return_data, ['TotalAssetsEOYAmt'], 0)
    
    #TotalLiabilitiesEOYAmt
    TotalLiabilitiesEOYAmt = get_field(return_data, ['TotalLiabilitiesEOYAmt'], 0)
    
    #NetAssetsOrFundBalancesEOYAmt
    NetAssetsOrFundBalancesEOYAmt = get_field(return_data, ['NetAssetsOrFundBalancesEOYAmt'], 0)

    #ActivityOrMissionDesc
    ActivityOrMissionDesc = get_field(return_data, ['ActivityOrMissionDesc'], "")
    
    #MissionDesc
    descri = get_field(return_data, ['MissionDesc'], "")

    #TypeOfOrganizationCorpInd
    corp_ind = get_field(return_data, ['TypeOfOrganizationCorpInd'], "NA")
    
    #Trust
    TypeOfOrganizationTrustInd = get_field(return_data, ['TypeOfOrganizationTrustInd'], "NA")
    
    #asso
    TypeOfOrganizationAssocInd = get_field(return_data, ['TypeOfOrganizationAssocInd'], "NA")
    
    #other
    TypeOfOrganizationOtherInd = get_field(return_data, ['TypeOfOrganizationOtherInd'], "NA")
    
    #SchoolOperatingInd
    SchoolOperatingInd = get_field(return_data, ['SchoolOperatingInd'], "NA")
    
    #OperateHospitalInd
    OperateHospitalInd = get_field(return_data, ['OperateHospitalInd'], "NA") #Sometimes has {'referenceDocumentId': 'IRS990ScheduleH'} for true 
    
    #AddressChangeInd
    AddressChangeInd = get_field(return_data, ['AddressChangeInd'], "")
    
    #InitialReturnInd
    InitialReturnInd = get_field(return_data, ['InitialReturnInd'], "")
    
    #AmendedReturnInd
    AmendedReturnInd = get_field(return_data, ['AmendedReturnInd'], "")
    
    #TerminateOperationsInd
    TerminateOperationsInd = get_field(return_data, ['TerminateOperationsInd'], "")
    
    #ScheduleO
    schedule0 = get_field(xmldict,['ReturnData','IRS990ScheduleO'], "none")
   


    return_list = [[formation_year, legal_dom, related, gov_grants,fed_camp,membership,fundraising, NoncashContributionsAmt,all_other, total_contri,
                          program_service_rev, CY_investment_income,CYOtherRevenueAmt, CYTotalRevenueAmt,CYTotalExpensesAmt,CYRevenuesLessExpensesAmt,
                          TotalAssetsEOYAmt,TotalLiabilitiesEOYAmt,NetAssetsOrFundBalancesEOYAmt,ActivityOrMissionDesc, descri, corp_ind,
                        TypeOfOrganizationTrustInd, TypeOfOrganizationAssocInd, TypeOfOrganizationOtherInd, SchoolOperatingInd, OperateHospitalInd,
                         AddressChangeInd, InitialReturnInd,AmendedReturnInd,TerminateOperationsInd,schedule0]]
    
    return_df = pd.DataFrame(return_list, columns = ['FormationYr', 'LegalDomicileStateCd','RelatedOrganizationsAmt','GovernmentGrantsAmt',
                                                                 'FederatedCampaignsAmt','MembershipDuesAmt','FundraisingAmt','NoncashContributionsAmt',
                                                                 'AllOtherContributionsAmt','TotalContributionsAmt','TotalProgramServiceRevenueAmt','CYInvestmentIncomeAmt',
                                                                 'CYOtherRevenueAmt','CYTotalRevenueAmt','CYTotalExpensesAmt','CYRevenuesLessExpensesAmt','TotalAssetsEOYAmt',
                                                                 'TotalLiabilitiesEOYAmt','NetAssetsOrFundBalancesEOYAmt',
                                                                 'ActivityOrMissionDesc','MissionDesc','TypeOfOrganizationCorpInd','TypeOfOrganizationTrustInd','TypeOfOrganizationAssocInd',
                                                                 'TypeOfOrganizationOtherInd','SchoolOperatingInd',
                                                                 'OperateHospitalInd','AddressChangeInd', 'InitialReturnInd','AmendedReturnInd','TerminateOperationsInd', 'schedule0'])
    ##############Schedule O
    #scheduleodesc = get_field(xmldict, ['ReturnData','ScheduleO'])
    
    #FORM 990, PART VI, SECTION B, LINE 11B
    #Form 990, Part III, Line 1
    
    #Concatenating header information and contributions
    return pd.concat([object_df, filer_df, return_df], axis=1, ignore_index=False)
    

### Example Analysis


In [9]:
df = pd.DataFrame([])

one_row = analyze_one('C:/Users/Administrator/Desktop/irs_2017/201743569349100404_public.xml')
df = df.append(one_row) 
df

""


### Get a list of files to analyze
You can generate a full index at the path used below with `extract_eins.ipynb`, or just download [this one](https://www.dropbox.com/s/237lcs9dcmqzjug/full_index.csv?dl=0) and place it in a folder called `index`. 

In [5]:
index = pd.read_csv('index/full_index.csv')
files = 'data/' + index.loc[index['TAX_YEAR'] == 2016]['OBJECT_ID'].astype('str') + '_public.xml'
files

6          data/201731359349202698_public.xml
15         data/201800529349300205_public.xml
19         data/201721359349307007_public.xml
28         data/201701869349300730_public.xml
36         data/201702209349300965_public.xml
                          ...                
3261613    data/201741309349201189_public.xml
3261621    data/201713109349302626_public.xml
3261629    data/201703189349103250_public.xml
3261637    data/201841319349304634_public.xml
3261645    data/201733199349101418_public.xml
Name: OBJECT_ID, Length: 437222, dtype: object

If you do not have all of these files available, you can do the following:

1. Run `files.to_csv('index/2017.txt', header=False, index=False)` to get a list of all the files you need to download
2. Run this in powershell: `ForEach ($file in (Get-Content -Path "index/2017.txt")) {aws s3 cp ("s3://irs-form-990/"+$file) ("data/"+$file)}` 

Note: I'm not able to completely test this script without reinstalling aws so this hasn't been completely tested but it *should* work.


### Alternative using all the files in the directory 

In [5]:
#This is the method I used to get all the files in the directory

import os

'''
    For the given path, get the List of all files in the directory tree 
'''
def getListOfFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles


dirName = 'C:/Users/Administrator/Desktop/irs_2017';
files = getListOfFiles(dirName)






### Create the dataframe with selected fields

In [6]:
df = pd.concat([one_row for name in tqdm(files) if (one_row := analyze_one(name)) is not None])
df

100%|██████████| 428974/428974 [1:02:20<00:00, 114.68it/s]


,Object_ID,TaxYr,TaxPeriodBeginDt,EIN,BusinessName,City,State,ZIPCd,FormationYr,LegalDomicileStateCd,...,TypeOfOrganizationTrustInd,TypeOfOrganizationAssocInd,TypeOfOrganizationOtherInd,SchoolOperatingInd,OperateHospitalInd,AddressChangeInd,InitialReturnInd,AmendedReturnInd,TerminateOperationsInd,schedule0
0,2017201700069349300000,2014,2014-01-01,208368409,THE FAMILY CENTER OF GAP INC,Gap,PA,17527,2007,PA,...,NA,NA,NA,false,false,X,,,false,{'{http://www.irs.gov/efile}SupplementalInform...
0,2017201700069349300005,2015,2015-07-01,710433583,FRIENDSHIP COMMUNITY CARE INC,RUSSELLVILLE,AR,72811,1975,AR,...,NA,NA,NA,false,false,,,,false,{'{http://www.irs.gov/efile}SupplementalInform...
0,2017201700069349300010,2015,2015-01-01,352420579,DETROIT HBCU NETWORK,Grosse Pointe,MI,48236,2013,MI,...,NA,NA,X,false,false,,X,,false,{'{http://www.irs.gov/efile}SupplementalInform...
0,2017201700069349300015,2015,2015-09-01,201597292,HOWELL BASKETBALL CLUB INC,FARMINGDALE,NJ,07727,2004,NJ,...,NA,NA,NA,false,false,,,,false,{'{http://www.irs.gov/efile}SupplementalInform...
0,2017201700069349300020,2015,2015-07-01,943121699,THE DAVIS STREET COMMUNITY CENTER,SAN LEANDRO,CA,94577,1990,CA,...,NA,NA,NA,false,false,,,,false,{'{http://www.irs.gov/efile}SupplementalInform...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2017201743579349300104,2016,2016-07-01,580611310,National Mental Health Association Georg,Atlanta,GA,30329,1976,GA,...,NA,NA,NA,false,false,X,,,false,{'{http://www.irs.gov/efile}SupplementalInform...
0,2017201743579349300504,2016,2016-04-01,421238299,IOWA ELKS ASSOCIATION CHARITABLE,MUSCATINE,IA,52761,1983,IA,...,NA,NA,NA,false,false,,,,false,{'{http://www.irs.gov/efile}SupplementalInform...
0,2017201743609349300104,2015,2015-07-01,465684634,MT EDEN HS FRIENDS OF THE CHOIR,Hayward,CA,94542,2010,CA,...,NA,NA,NA,false,false,,,,false,{'{http://www.irs.gov/efile}SupplementalInform...
0,2017201743609349300404,2016,2016-09-01,223780975,LITTLE FLYERS ACADEMY INC,ATLANTIC CITY AIRPORT,NJ,08405,2000,NJ,...,NA,NA,NA,false,false,,,,false,{'{http://www.irs.gov/efile}SupplementalInform...


In [7]:
df.to_csv("C:/Users/Administrator/Desktop/Impact_Capital/data/2017_objects.csv")

## stop here to add each year in parallel

# Data Cleaning

In [13]:
pd.options.display.max_columns = None
form990 = pd.read_csv("data/Form_990_2017.csv")
form990

,Unnamed: 0,Object_ID,TaxYr,TaxPeriodBeginDt,EIN,BusinessName,City,State,ZIPCd,FormationYr,LegalDomicileStateCd,RelatedOrganizationsAmt,GovernmentGrantsAmt,FederatedCampaignsAmt,MembershipDuesAmt,FundraisingAmt,NoncashContributionsAmt,AllOtherContributionsAmt,TotalContributionsAmt,TotalProgramServiceRevenueAmt,CYInvestmentIncomeAmt,CYOtherRevenueAmt,CYTotalRevenueAmt,CYTotalExpensesAmt,CYRevenuesLessExpensesAmt,TotalAssetsEOYAmt,TotalLiabilitiesEOYAmt,NetAssetsOrFundBalancesEOYAmt,ActivityOrMissionDesc,MissionDesc,TypeOfOrganizationCorpInd,TypeOfOrganizationTrustInd,TypeOfOrganizationAssocInd,TypeOfOrganizationOtherInd,SchoolOperatingInd,OperateHospitalInd,AddressChangeInd,InitialReturnInd,AmendedReturnInd,TerminateOperationsInd,schedule0
0,0,2018201800099349300835,2017,2017-01-01,471186799,LEDFORD PANTHER ATHLETIC ASSOC INC,THOMASVILLE,NC,27360.0,2015.0,NC,0,0,0,205,0,0,1200,1405,64754,0,6597,72756,64238,8518,30869,0,30869,HIGH SCHOOL ATHLETIC BOOSTER CLUB. PROVIDES VO...,HIGH SCHOOL ATHLETIC BOOSTER CLUB. PROVIDES VO...,X,NaN,NaN,NaN,false,false,NaN,NaN,NaN,false,{'{http://www.irs.gov/efile}SupplementalInform...
1,0,2018201800119349300625,2017,2017-01-01,200090779,FRIENDS OF THE LEEWARD COAST PUBLIC,KAPOLEI,HI,96707.0,2001.0,NaN,0,0,0,0,0,0,1233610,1233610,0,18,0,1233628,1005716,227912,910892,0,910892,EDUCATION,EDUCATION,X,NaN,NaN,NaN,false,false,NaN,NaN,NaN,false,{'{http://www.irs.gov/efile}SupplementalInform...
2,0,2018201800139349300710,2017,2017-01-01,431817270,WINONA VOLUNTEER FIRE DEPARTMENT,WINONA,MO,65588.0,1930.0,MO,0,0,0,0,0,0,50,50,0,31163,0,31213,27682,3531,1096269,82643,1013626,FIGHTING FIRES/EMERGENCY ASSI,FIGHTING FIRES/EMERGENCY ASSI,NaN,NaN,X,NaN,false,false,NaN,NaN,NaN,false,{'{http://www.irs.gov/efile}SupplementalInform...
3,0,2018201800179349300300,2017,2017-01-01,540943951,ANTIOCH RECREATION ASSOCIATION INC,SANDSTON,VA,23150.0,1995.0,VA,0,0,0,0,0,0,1551,1551,54283,4,21583,77421,81674,-4253,19520,3703,15817,PROVIDE A SAFE AND ENJOYABLE RECREATION FACILI...,PROVIDE A SAFE AND ENJOYABLE RECREATION FACILI...,X,NaN,NaN,NaN,false,false,NaN,NaN,NaN,false,{'{http://www.irs.gov/efile}SupplementalInform...
4,0,2018201800189349300540,2017,2017-01-01,356024706,GREATER LAFAYETTE KENNEL CLUB INC,Lafayette,IN,47903.0,1945.0,IN,0,0,0,2114,0,0,12337,14451,130129,27,0,144607,105828,38779,120732,0,120732,TO FURTHER THE ADVANCEMENT OF ALL BREEDS OF DO...,TO FURTHER THE ADVANCEMENT OF ALL BREEDS OF DO...,X,NaN,NaN,NaN,false,false,NaN,NaN,NaN,false,{'{http://www.irs.gov/efile}SupplementalInform...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253138,0,2019201943529349300814,2017,2017-08-01,330332711,CARE HOUSING SERVICES CORPORATION,TARZANA,CA,91356.0,NaN,NaN,0,20117,0,0,0,0,0,20117,858863,477,352476,1231933,1260716,-28783,750260,8815,741445,SEE ATTACHED STATEMENT #2,SEE ATTACHED STATEMENT #2,NaN,NaN,NaN,NaN,false,false,NaN,NaN,NaN,false,{'{http://www.irs.gov/efile}SupplementalInform...
253139,0,2019201943529349301324,2017,2017-10-01,680547196,Center for Restorative Justice Works,Montebello,CA,90640.0,2002.0,CA,0,0,0,0,0,31500,878007,878007,0,8064,0,886071,952202,-66131,1093731,26993,1066738,CJRW IS DEDICATED TO IMPROVING THE LIVES OF WO...,THE SPECIFIC PURPOSE OF THE ORGANIZATION SHALL...,X,NaN,NaN,NaN,false,false,X,NaN,NaN,false,{'{http://www.irs.gov/efile}SupplementalInform...
253140,0,2019201943539349300504,2017,2017-11-01,360865810,CALUMET COUNTRY CLUB,HOMEWOOD,IL,60430.0,1901.0,IL,0,0,0,768749,0,0,0,768749,275195,769232,113890,1927066,1863328,63738,0,2488791,-2488791,SOCIAL CLUB,SOCIAL CLUB,NaN,NaN,X,NaN,false,false,NaN,NaN,NaN,{'referenceDocumentId': 'RetDoc3'},{'{http://www.irs.gov/efile}SupplementalInform...
253141,0,2019201943539349301709,2017,2017-01-01,821672086,IN OUR OWN VOICE NATIONAL BLACK WOMENS,WASHINGTON,DC,20005.0,2017.0,DE,0,0,0,0,0,0,2950200,2950200,0,0,0,2950200,22259,2927941,2927941